In [6]:
%load_ext autoreload
%autoreload 2
import sys, os
sys.path.append(os.path.abspath("../../"))
from dotenv import load_dotenv
load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [17]:
from src.config import Config, load_spreadsheet
from src.rag import (
    ChromaSlideStore,
    HyperbolicScorer,
    MinScorer,
    PresentationRetriever,
    ScorerTypes,
)
from src.eval.eval_mlflow import EvaluationConfig, RAGEvaluator

import mlflow

In [18]:
# Mlflow setup logging
mlflow.langchain.autolog()

# Setup components
project_config = Config()
llm = project_config.model_config.load_vsegpt(model="openai/gpt-4o-mini")
embeddings = project_config.embedding_config.load_vsegpt()
embeddings = project_config.embedding_config.load_openai()

storage = ChromaSlideStore(collection_name="pres0", embedding_model=embeddings)

db_path = project_config.navigator.processed / "eval" / "runs" / "mlruns.db"
artifacts_path = project_config.navigator.processed / "eval" / "artifacts"
eval_config = EvaluationConfig(
    experiment_name="PresRetrieve_noanswer",
    metrics=["presentationmatch", "llmrelevance"],
    scorers=[MinScorer()],

)

evaluator = RAGEvaluator(
    storage=storage,
    config=eval_config,
    llm=llm
)

In [19]:
from IPython.display import display
# Load questions
sheet_id = os.environ["BENCHMARK_SPREADSHEET_ID"]
gids = {
    "NoAnswer": "1219206941"
}
df = evaluator.load_questions_from_sheet(sheet_id, gid=gids["NoAnswer"])

df_eval = df.sample(5)
df_eval = df.copy()
display(df_eval)

,pres_name,question,page,content,comment
0,NaN,Презентация про космонавтов,,NaN,NaN
1,NaN,Презентация про экономику Китая,,NaN,NaN
2,NaN,Зоомагазины,,NaN,NaN
3,NaN,The capital of Great Britain,,NaN,NaN
4,NaN,Обучение LLM на CPU,,NaN,NaN
5,NaN,Фото кабриолета,,NaN,NaN
6,NaN,История Российской Империи,,NaN,NaN


In [20]:

# Run evaluation
evaluator.run_evaluation(df_eval)

